In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# def range of yellow color HSV color space
lower_yellow = np.array([20, 0, 0])
upper_yellow = np.array([43,255, 255])


# array for storing image center points for tracking op
points = []

# get def cam window size
ret, frame = cap.read()
height, width = frame.shape[:2]
frame_count = 0

while 1:
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # thresh hsv image to get only yellow colors
    mask = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    contours, heirachy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # creates empty center array to store centroid and center of mass
    center = int(height / 2), int(width / 2)
    
    if len(contours) > 0:
        # gets largest contour only and its center
        contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(contour)     # boudning cicle round the detected contour
        M = cv2.moments(contour)
        try:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        except:
            center = int(height/2), int(width/2)     # resets center back
            
        if(radius > 25):  # true if radius of enclosing circle of contour is > 25
            # draw circle and leave last center creating a trail
            cv2.circle(frame, (int(x), int(y)), int(radius), (0,0, 255), 2)
            cv2.circle(frame, center, 5, (0, 255, 0), -1)
            
    # logs center points for tracking opp
    points.append(center)
    
    # iterate over each trackpoint and draw lines over each forming tracking line
    if radius > 25:
        for i in range(1, len(points)):
            try:
                cv2.line(frame, points[i -1], points[i], (0, 0, 255), 2)
            except:
                pass
            
        # make frame count zero
        frame_count = 0
    else:
        # updates frame_count
        frame_count += 1
        
        # if no frame_count = 6 and obj not detectec we del trail
        if frame_count == 10:
            points.clear()
            frame_count = 0
            
    # display the simple obj Tracker
    frame = cv2.flip(frame, 1)
    cv2.imshow("Simple Object Tracker", frame)
    
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()